In [80]:
import chess
import pandas as pd
from tqdm import tqdm

In [81]:
games = pd.read_csv('games.csv')

In [3]:
def extract_dictionary(sequence):
    chessboard = {
             'wr-L': 'a1', 'wn-L': 'b1', 'wb-L': 'c1', 'wq':   'd1', 'wk':   'e1', 'wb-R': 'f1', 'wn-R': 'g1', 'wr-R': 'h1',
              'wp-a': 'a2', 'wp-b': 'b2', 'wp-c': 'c2', 'wp-d': 'd2', 'wp-e': 'e2', 'wp-f': 'f2', 'wp-g': 'g2', 'wp-h': 'h2',
              'bp-a': 'a7', 'bp-b': 'b7', 'bp-c': 'c7', 'bp-d': 'd7', 'bp-e': 'e7', 'bp-f': 'f7', 'bp-g': 'g7', 'bp-h': 'h7',
              'br-L': 'a8', 'bn-L': 'b8', 'bb-L': 'c8', 'bq':   'd8', 'bk':   'e8', 'bb-R': 'f8', 'bn-R': 'g8', 'br-R': 'h8'
        }

    for piece, move in chessboard.items():
        chessboard[piece] = [move]

    board = chess.Board()
    for t in sequence:
        s = str(board.push_san(t))
        before = s[0:2]
        after = s[2:]
        for piece, moves in chessboard.items():
            if moves[-1] == before:
                chessboard[piece] = moves + [after]
            else: 
                chessboard[piece] = moves + [moves[-1]]
    return chessboard

In [66]:
def extract_moves_openings(sequence):
    result = []
    chessboard = {
         'wr-L': 'a1', 'wn-L': 'b1', 'wb-L': 'c1', 'wq':   'd1', 'wk':   'e1', 'wb-R': 'f1', 'wn-R': 'g1', 'wr-R': 'h1',
          'wp-a': 'a2', 'wp-b': 'b2', 'wp-c': 'c2', 'wp-d': 'd2', 'wp-e': 'e2', 'wp-f': 'f2', 'wp-g': 'g2', 'wp-h': 'h2',
          'bp-a': 'a7', 'bp-b': 'b7', 'bp-c': 'c7', 'bp-d': 'd7', 'bp-e': 'e7', 'bp-f': 'f7', 'bp-g': 'g7', 'bp-h': 'h7',
          'br-L': 'a8', 'bn-L': 'b8', 'bb-L': 'c8', 'bq':   'd8', 'bk':   'e8', 'bb-R': 'f8', 'bn-R': 'g8', 'br-R': 'h8'
    }
    
    for piece, move in chessboard.items():
        chessboard[piece] = [move]

    board = chess.Board()
    for t in sequence:
        s = str(board.push_san(t))
        before = s[0:2]
        after = s[2:]
        for piece, moves in chessboard.items():
            if moves[-1] == before:
                chessboard[piece] = moves + [after]
                result.append([piece, after])
    return result

In [128]:
def get_piece(board, pos):
    for piece, p in board.items():
        if p == pos:
            return piece

def extract_states(sequence):
    chessboard = {
        'wr-L': 'a1', 'wn-L': 'b1', 'wb-L': 'c1', 'wq':   'd1', 'wk':   'e1', 'wb-R': 'f1', 'wn-R': 'g1', 'wr-R': 'h1',
        'wp-a': 'a2', 'wp-b': 'b2', 'wp-c': 'c2', 'wp-d': 'd2', 'wp-e': 'e2', 'wp-f': 'f2', 'wp-g': 'g2', 'wp-h': 'h2',
        'bp-a': 'a7', 'bp-b': 'b7', 'bp-c': 'c7', 'bp-d': 'd7', 'bp-e': 'e7', 'bp-f': 'f7', 'bp-g': 'g7', 'bp-h': 'h7',
        'br-L': 'a8', 'bn-L': 'b8', 'bb-L': 'c8', 'bq':   'd8', 'bk':   'e8', 'bb-R': 'f8', 'bn-R': 'g8', 'br-R': 'h8'
    }
    engine = chess.Board()
    
    print(sequence)
    
    states = [chessboard.copy()]
    for move in sequence:
        s = str(engine.push_san(move))
        old_pos = s[0:2]
        new_pos = s[2:]        
            
        if len(new_pos) == 2:
            
            if move == 'O-O':
                if engine.turn: # black move
                    chessboard['br-R'] = 'f8'
                else:
                    chessboard['wr-L'] = 'd1'
            if move == 'O-O-O':
                if engine.turn: # black move
                    chessboard['br-L'] = 'd8'
                else:
                    chessboard['wr-R'] = 'f1'
            
            captured = get_piece(chessboard, new_pos)
            if captured is not None:
                del chessboard[captured]
            chessboard[get_piece(chessboard, old_pos)] = new_pos
            
            states.append(chessboard.copy())
        else:
            print(s, new_pos)
    return states

In [129]:
openings = pd.read_json('data/openings.json').T
openings.head(5)

,winner_white,winner_black,draw,opening_moves,states,nb_games,rating_histogram
Sicilian Defense,45,49,127,"[Nc3, c5, e4, Nc6]","[{'wr-L': 'a1', 'wn-L': 'c3', 'wb-L': 'c1', 'w...",2573,"[789, 826, 845, 857, 857, 871, 874, 880, 886, ..."
French Defense,49,45,65,"[Nc3, Nf6, d4, d5, Bg5, e6, e4, Be7, e5, Nfd7,...","[{'wr-L': 'a1', 'wn-L': 'c3', 'wb-L': 'c1', 'w...",1306,"[795, 904, 906, 921, 925, 928, 930, 937, 940, ..."
Queen's Pawn Game,46,48,49,"[Nc3, Nf6, d4, d5]","[{'wr-L': 'a1', 'wn-L': 'c3', 'wb-L': 'c1', 'w...",1059,"[793, 820, 830, 835, 845, 856, 860, 905, 907, ..."
Italian Game,49,45,47,"[Nh3, e5, e4, Nc6, Bc4, Nf6, Ng5, d5]","[{'wr-L': 'a1', 'wn-L': 'b1', 'wb-L': 'c1', 'w...",981,"[886, 914, 921, 924, 932, 951, 963, 970, 972, ..."
King's Pawn Game,47,48,36,"[Nc3, e5, e4, Nf6, f3]","[{'wr-L': 'a1', 'wn-L': 'c3', 'wb-L': 'c1', 'w...",917,"[784, 800, 801, 807, 807, 807, 813, 828, 837, ..."


In [133]:
openings["opening_moves"]

Sicilian Defense                                          [Nc3, c5, e4, Nc6]
French Defense             [Nc3, Nf6, d4, d5, Bg5, e6, e4, Be7, e5, Nfd7,...
Queen's Pawn Game                                         [Nc3, Nf6, d4, d5]
Italian Game                           [Nh3, e5, e4, Nc6, Bc4, Nf6, Ng5, d5]
King's Pawn Game                                      [Nc3, e5, e4, Nf6, f3]
Ruy Lopez                  [Nc3, Nf6, Nf3, Nc6, e4, e5, Bb5, d6, d4, Bd7,...
English Opening                                      [Nf3, Nc6, c4, e5, Nc3]
Scandinavian Defense                                            [b4, d5, e4]
Caro-Kann Defense                                          [Nc3, c6, e4, d5]
Scotch Game                                           [Nf3, Nc6, e4, e5, d4]
Queen's Gambit Declined    [Nf3, Nf6, d4, d5, Bg5, e6, e3, Be7, c4, O-O, ...
Four Knights Game                     [Nc3, Nf6, e4, e5, d4, Nc6, Nf3, exd4]
Van't Kruijs Opening                                                    [e3]

In [130]:
openings['states'] = openings.opening_moves.apply(lambda x: extract_states(x))

['Nc3', 'c5', 'e4', 'Nc6']
['Nc3', 'Nf6', 'd4', 'd5', 'Bg5', 'e6', 'e4', 'Be7', 'e5', 'Nfd7', 'Bxe7', 'Qxe7', 'f4']
['Nc3', 'Nf6', 'd4', 'd5']
['Nh3', 'e5', 'e4', 'Nc6', 'Bc4', 'Nf6', 'Ng5', 'd5']
['Nc3', 'e5', 'e4', 'Nf6', 'f3']
['Nc3', 'Nf6', 'Nf3', 'Nc6', 'e4', 'e5', 'Bb5', 'd6', 'd4', 'Bd7', 'Bxc6']
['Nf3', 'Nc6', 'c4', 'e5', 'Nc3']
['b4', 'd5', 'e4']
['Nc3', 'c6', 'e4', 'd5']
['Nf3', 'Nc6', 'e4', 'e5', 'd4']
['Nf3', 'Nf6', 'd4', 'd5', 'Bg5', 'e6', 'e3', 'Be7', 'c4', 'O-O', 'Nc3']
['Nc3', 'Nf6', 'e4', 'e5', 'd4', 'Nc6', 'Nf3', 'exd4']
['e3']
['Nc3', 'd5', 'Nf3']
['Nc3', 'e5', 'e4', 'Nf6', 'Bc4', 'Nc6', 'd3']
['Nf3', 'Nf6', 'd4', 'c5']
['d4', 'e5', 'Nf3', 'd6', 'e4']
['Nf3', 'd5', 'd4', 'Nf6', 'c4', 'dxc4', 'e3']
['Nc3', 'e5', 'e4', 'Nf6', 'Nf3']
['Nf3', 'Nc6', 'd4', 'd5', 'c4', 'Nf6', 'Nc3', 'dxc4']
['Nc3', 'g6', 'e4', 'Bg7', 'd4', 'd6']
['d4', 'e6']
['d4', 'Nc6', 'e4', 'e5']
['Nf3', 'd6', 'e4', 'e5', 'Bc4']
['Nf3', 'Nf6', 'c4', 'd6', 'Nc3', 'g6', 'd4', 'Bg7', 'g3', 'O-O', 'Bg2', '

In [131]:
openings[['winner_white', 'winner_black', 'draw', 'opening_moves', 'states', 'nb_games', 'rating_histogram']].to_json('data/openings.json', orient = 'index')

In [90]:
total_results = {}
chessboard = {
     'wr-L': 'a1', 'wn-L': 'b1', 'wb-L': 'c1', 'wq':   'd1', 'wk':   'e1', 'wb-R': 'f1', 'wn-R': 'g1', 'wr-R': 'h1',
      'wp-a': 'a2', 'wp-b': 'b2', 'wp-c': 'c2', 'wp-d': 'd2', 'wp-e': 'e2', 'wp-f': 'f2', 'wp-g': 'g2', 'wp-h': 'h2',
      'bp-a': 'a7', 'bp-b': 'b7', 'bp-c': 'c7', 'bp-d': 'd7', 'bp-e': 'e7', 'bp-f': 'f7', 'bp-g': 'g7', 'bp-h': 'h7',
      'br-L': 'a8', 'bn-L': 'b8', 'bb-L': 'c8', 'bq':   'd8', 'bk':   'e8', 'bb-R': 'f8', 'bn-R': 'g8', 'br-R': 'h8'
}
list_pieces = chessboard.keys()
for p in list_pieces:
    total_results[p] = []

In [91]:
for i, row in games.iterrows():
    res = extract_dictionary(row.moves.split(" "))
    elo = int(0.5 * row.white_rating + 0.5 * row.black_rating)
    for piece, lst in total_results.items():
        total_results[piece] = lst + [[elo, res[piece]]]

In [95]:
flat_list = [item for sublist in total_results['wr-L'] for item in sublist]

In [116]:
for p in tqdm(list_pieces):
    temp = []
    for el in total_results[p]:
        temp.append([el[0]] + [i for i in el[1]])
    temp_df = pd.DataFrame(temp)
    temp_df = temp_df.rename({0: 'ELO'}, axis = 1)
    temp_df.to_pickle(f'flows/{p}.pkl')

100%|██████████| 32/32 [01:00<00:00,  1.90s/it]


In [118]:
pd.read_pickle('flows/bb-L.pkl').set_index('ELO').to_json('bb-L.json', )

,1,2,3,4,5,6,7,8,9,10,...,341,342,343,344,345,346,347,348,349,350
ELO,,,,,,,,,,,,,,,,,,,,,
1345,c8,c8,c8,c8,c8,c8,c8,c8,c8,c8,...,None,None,None,None,None,None,None,None,None,None
1291,c8,c8,c8,c8,c8,c8,c8,c8,c8,c8,...,None,None,None,None,None,None,None,None,None,None
1498,c8,c8,c8,c8,c8,c8,c8,c8,c8,c8,...,None,None,None,None,None,None,None,None,None,None
1446,c8,c8,c8,c8,f5,f5,f5,f5,f5,f5,...,None,None,None,None,None,None,None,None,None,None
1496,c8,c8,c8,c8,c8,c8,c8,c8,c8,c8,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,c8,c8,c8,c8,c8,c8,c8,c8,c8,c8,...,None,None,None,None,None,None,None,None,None,None
1214,c8,c8,c8,c8,c8,c8,c8,c8,c8,c8,...,None,None,None,None,None,None,None,None,None,None
1252,c8,c8,c8,c8,c8,c8,c8,c8,c8,c8,...,None,None,None,None,None,None,None,None,None,None


In [109]:
temp = []
for el in total_results['wr-L']:
    temp.append([el[0]] + [i for i in el[1]])
temp_df = pd.DataFrame(temp)
temp_df = temp_df.rename({0: 'ELO'}, axis = 1)

,ELO,1,2,3,4,5,6,7,8,9,...,341,342,343,344,345,346,347,348,349,350
0,1345,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
1,1291,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
2,1498,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
3,1446,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
4,1496,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,1455,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
20054,1214,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
20055,1252,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
20056,1293,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None


In [112]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,341,342,343,344,345,346,347,348,349,350
0,1345,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
1,1291,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
2,1498,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
3,1446,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
4,1496,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20053,1455,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
20054,1214,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
20055,1252,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
20056,1293,a1,a1,a1,a1,a1,a1,a1,a1,a1,...,None,None,None,None,None,None,None,None,None,None
